# Crewai + Personalised Educational Assistant

#### Refered documentation: 
#### https://docs.crewai.com/en/guides/crews/first-crew
#### https://docs.crewai.com/en/learn/create-custom-tools
#### https://docs.crewai.com/en/tools/search-research/serperdevtool
#### https://serper.dev/

In [72]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool
from pydantic import BaseModel
from typing import List
import os
from openai import OpenAI
import json
from crewai.tools import tool

In [17]:
os.environ["SERPER_API_KEY"] = "<API KEY>"
os.environ["OPENAI_API_KEY"] = "<API KEY>"

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

In [74]:
class LearningMaterialsModel(BaseModel):
    articles: List[str] 
    article_links: List[str]    
    video_links: List[str]       
    exercises: List[str]     

class QuizQuestion(BaseModel):
    question: str
    options: List[str]
    answer: str


class QuizModel(BaseModel):
    questions: List[QuizQuestion]


class ProjectIdeasModel(BaseModel):
    ideas: List[str]

In [76]:
client = OpenAI()

@tool("project_suggestion_tool")
def project_suggestion_tool(topic: str, expertise_level: str) -> str:
    """
    Uses OpenAI LLM to generate realistic project ideas for the given topic
    and expertise level. Returns a JSON string containing the ideas list.
    """
    prompt = f"""
    Generate 4 practical, high-quality project ideas for a learner studying the topic:
    "{topic}"

    Their expertise level is: {expertise_level}.

    Requirements:
    - The project ideas MUST be realistic, implementable, and topic-specific.
    - Avoid generic suggestions.
    - Output ONLY a JSON list of strings, example:
      ["idea1", "idea2", "idea3", "idea4"]
    """

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
    )

    ideas_raw = response.choices[0].message.content.strip()

    try:
        ideas = json.loads(ideas_raw)
    except:
        ideas = [ideas_raw]

    return json.dumps({"ideas": ideas})

In [78]:
serper_tool = SerperDevTool()

In [80]:
learning_material_agent = Agent(
    role="Learning Material Agent",
    goal="Curate article summaries, extract article links and YouTube video links, and generate exercises for user topics.",
    backstory=(
        "An expert educational content curator trained in extracting high-quality articles "
        "from the web and generating meaningful practice exercises."
    ),
    tools=[serper_tool],
    allow_delegation=False,
    verbose=True,
)

quiz_agent = Agent(
    role="Quiz Creator Agent",
    goal="Generate personalised quizzes to evaluate user understanding.",
    backstory=(
        "A specialized assessment designer with experience in pedagogy and question formulation."
    ),
    tools=[],
    allow_delegation=False,
    verbose=True,
)

project_agent = Agent(
    role="Project Idea Agent",
    goal="Recommend project ideas tailored to the user's expertise level and topic.",
    backstory=(
        "A mentor-like agent that helps learners practice real-world skills through hands-on projects."
    ),
    tools=[project_suggestion_tool],
    allow_delegation=False,
    verbose=True,
)

In [82]:
materials_task = Task(
    description=(
        "Generate structured learning materials for the topic: {topic}." 
        "Use SERPER to extract:"
        "- High-quality ARTICLE LINKS (actual URLs)"
        "- YOUTUBE VIDEO LINKS (actual URLs)"
        "Also:"
        "- Produce ARTICLE SUMMARIES by synthesizing content from the top articles."
        "- Generate EXERCISES based on the topic."
        "Return output in LearningMaterialsModel JSON format:"
        "{articles: [...], article_links: [...], video_links: [...], exercises: [...]}"
        "IMPORTANT:"
        "- Extract the 'link' field from Serper results, not snippets."
        "- For video_links, include only links containing 'youtube.com' or 'youtu.be'."
    ),
    agent=learning_material_agent,
    expected_output="A JSON matching LearningMaterialsModel: {articles: [...],article_links: [...], video_links: [...], exercises: [...]}",
    output_json=LearningMaterialsModel,
)


quiz_task = Task(
    description=(
        "Create a personalized quiz for topic: {topic}. "
        "Generate 5–10 questions including multiple-choice and conceptual questions. "
        "Return output following QuizModel schema."
    ),
    agent=quiz_agent,
    expected_output="A JSON matching QuizModel: {questions: [...]}.",
    output_json=QuizModel,
)


project_task = Task(
    description=(
        "Suggest practical project ideas for the topic: {topic} "
        "and expertise level: {expertise_level}. "
        "Use the project_suggestion_tool to generate ideas. "
        "Return output following ProjectIdeasModel."
    ),
    agent=project_agent,
    expected_output="A JSON matching ProjectIdeasModel: {ideas: [...]}.",
    output_json=ProjectIdeasModel,
)

In [84]:
learning_assistant_crew = Crew(
    agents=[learning_material_agent, quiz_agent, project_agent],
    tasks=[materials_task, quiz_task, project_task],
    process=Process.sequential,
    verbose=True,
)

In [86]:
def run_learning_assistant(topic: str, expertise_level: str):
    inputs = {
        "topic": topic,
        "expertise_level": expertise_level,
    }

    result = learning_assistant_crew.kickoff(inputs=inputs)
    return result

### The customization/personalization comes here where the topic and difficulty level can be choosen here - it can also be obtained as an input from the user easily

In [89]:
result = run_learning_assistant("Sacoglossan sea slugs", "intermediate")
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 9c4415b4-0443-437b-aaa2-62e2777f674a                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Learning Material Agent                                                                                 │
│                                                                                                                 │
│  Task: Generate structured learning materials for the topic: Sacoglossan sea slugs.Use SERPER to extract:-      │
│  High-quality ARTICLE LINKS (actual URLs)- YOUTUBE VIDEO LINKS (actual URLs)Also:- Produce ARTICLE SUMMARIES    │
│  by synthesizing content from the top articles.- Generate EXERCISES based on the topic.Return output in         │
│  LearningMaterialsModel JSON format:{articles: [...], article_links: [...], video_links: [...], exercises:      │
│  [...]}IMPORTANT:- Extract the 'link' field from Serper results, not snippets.- For video_links, include only   │
│  links containing 'youtube.com' or 'youtu.be'.                                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Learning Material Agent                                                                                 │
│                                                                                                                 │
│  Thought: Thought: I need to search for high-quality articles on Sacoglossan sea slugs and extract relevant     │
│  links. Then I need to find YouTube videos related to Sacoglossan sea slugs. After gathering links, I will      │
│  synthesize summaries from the top articles and generate exercises related to the topic.                        │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Project Idea Agent                                                                                      │
│                                                                                                                 │
│  Thought: Thought: I need to generate practical project ideas tailored to the topic "Sacoglossan sea slugs"     │
│  and expertise level "intermediate" using the project_suggestion_tool.                                          │
│                                                                                                                 │
│  Using Tool: project_suggestion_tool                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Project Idea Agent                                                                                      │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {"ideas": ["Conduct a survey of local coastal ecosystems to document the diversity of Sacoglossan sea slugs,   │
│  focusing on their habitat preferences and associated algae species.", "Create a detailed photographic field    │
│  guide to the Sacoglossan sea slugs found in a specific region, including identification tips, habitat          │
│  descriptions, and ecological roles.", "Investigate the chemical ecology of Sacoglossan sea slugs by analyzing  │
│  the secondary metabolites from their algal food sources and their potential effects on predator deterrence.",  │
│  "Design an aquarium experiment to observe the feeding behavior of Sacoglossan sea slugs on different algal     │
│  species, measuring growth rates and survival in relation to dietary preferences."]}                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

{'ideas': ['Conduct a survey of local coastal ecosystems to document the diversity of Sacoglossan sea slugs, focusing on their habitat preferences and associated algae species.', 'Create a detailed photographic field guide to the Sacoglossan sea slugs found in a specific region, including identification tips, habitat descriptions, and ecological roles.', 'Investigate the chemical ecology of Sacoglossan sea slugs by analyzing the secondary metabolites from their algal food sources and their potential effects on predator deterrence.', 'Design an aquarium experiment to observe the feeding behavior of Sacoglossan sea slugs on different algal species, measuring growth rates and survival in relation to dietary preferences.']}


╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: b22d5898-d195-4935-9aa9-deb7f09cb690                                                                     │
│  Agent: Project Idea Agent                                                                                      │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [93]:
def print_learning_assistant_output(result):
    print("\n" + "="*80)
    print("📘 LEARNING MATERIALS")
    print("="*80)

    # ---- Learning Materials ----
    learning_data = result.tasks_output[0].json_dict
    
    article_summaries = learning_data.get("articles", [])
    article_links = learning_data.get("article_links", [])
    video_links = learning_data.get("video_links", [])
    exercises = learning_data.get("exercises", [])

    # --- Article Summaries ---
    print("\n🔹 ARTICLE SUMMARIES:")
    for i, summary in enumerate(article_summaries, 1):
        print(f"\n  {i}. {summary}")

    # --- Article Links ---
    print("\n🔹 ARTICLE LINKS:")
    for i, link in enumerate(article_links, 1):
        print(f"  {i}. {link}")

    # --- Video Links ---
    print("\n🔹 VIDEO LINKS:")
    for i, vlink in enumerate(video_links, 1):
        print(f"  {i}. {vlink}")

    # --- Exercises ---
    print("\n🔹 EXERCISES:")
    for i, ex in enumerate(exercises, 1):
        print(f"\n  {i}. {ex}")

    print("\n" + "="*80)
    print("📝 QUIZ QUESTIONS")
    print("="*80)

    # ---- Quiz Questions ----
    quiz_data = result.tasks_output[1].json_dict
    questions = quiz_data.get("questions", [])

    for i, q in enumerate(questions, 1):
        print(f"\nQ{i}. {q['question']}")
        for opt_i, opt in enumerate(q['options'], 1):
            print(f"   {opt_i}) {opt}")
        print(f"   ✔ Answer: {q['answer']}")

    print("\n" + "="*80)
    print("💡 PROJECT IDEAS")
    print("="*80)

    # ---- Project Ideas ----
    project_data = result.tasks_output[2].json_dict
    ideas = project_data.get("ideas", [])

    for i, idea in enumerate(ideas, 1):
        print(f"\n  {i}. {idea}")

    print("\n" + "="*80)


In [95]:
print_learning_assistant_output(result)


📘 LEARNING MATERIALS

🔹 ARTICLE SUMMARIES:

  1. Sacoglossan sea slugs are a unique group of marine gastropods that feed by sucking the cellular contents of algae, hence they are sometimes called sap-sucking sea slugs. They belong to the clade Heterobranchia and are known for their ability to incorporate chloroplasts from algae into their own tissues – a process called kleptoplasty. This allows some species to perform photosynthesis, providing them with energy from sunlight for extended periods without eating.

  2. Kleptoplasty is a remarkable biological phenomenon where sacoglossan sea slugs sequester functional chloroplasts from the algae they consume. These chloroplasts can remain active inside the slugs’ cells for weeks or even months, enabling the slugs to derive energy directly from photosynthesis. This ability has fascinated scientists due to its rarity and complexity, providing insights into symbiotic relationships and cell biology.

  3. Certain sacoglossan species, such as 